# Review DPS outputs
Make a mosaic of DPS outputs.

1. make a list of the DPS output paths with build_tindex.master.py
2. Identify duplicate tiles
3. Identify matching tiles; merge tindex.master with the original index tile file
4. make mosaicjson
5. View DPS results

In [1]:
import geopandas
import pandas as pd
import os
import json
import collections
import numpy as np
import sys
sys.path.append('/projects/icesat2_boreal/lib')
sys.path.append('/projects/Developer/icesat2_boreal/lib')
import maplib_folium
def local_to_s3(url, user = 'nathanmthomas', type='public'):
    ''' A Function to convert local paths to s3 urls'''
    if type is 'public':
        replacement_str = f's3://maap-ops-workspace/shared/{user}'
    else:
        replacement_str = f's3://maap-ops-workspace/{user}'
    return url.replace(f'/projects/my-{type}-bucket', replacement_str)

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# Choose a DPS data type; get its tile index master file
Get a filename of the master list DPS output tiles returned with _build_tindexmaster.py

In [2]:
DPS_DATA_TYPE = 'ATL08_filt' #"Topo" "Landsat" "ATL08" "ATL08_filt" "AGB"
DPS_DATA_USER = 'nathanmthomas' #'nathanmthomas' 'lduncanson'
tindex_master_fn = f'/projects/shared-buckets/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master.csv'
####---TMP---#####
tindex_master_fn = '/projects/my-private-bucket/dps_output/run_tile_atl08_ubuntu/master/2022/run_LC_height_thresholds_v2/ATL08_filt_tindex_master.csv'
#tindex_master_fn = f'/projects/shared-buckets/{DPS_DATA_USER}/Topo/{DPS_DATA_TYPE}_tindex_master.csv'
####

#tindex_master_fn = '/projects/my-private-bucket/dps_output/run_tile_atl08_ubuntu/master/2022/run_LC_height_thresholds/ATL08_filt_tindex_master.csv'

#Get other tile index master files

ATL08_filt_sample_tindex_master_fn = f'/projects/shared-buckets/lduncanson/DPS_tile_lists/ATL08_filt_sample_tindex_master.csv'
#ATL08_filt_tindex_master_fn = '/projects/test_dps/ATL08_filt_tindex_master.csv'#'s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv'
ATL08_filt_tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv'
Topo_tindex_master_fn = f'/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv'

Topo_tindex_master =   pd.read_csv(Topo_tindex_master_fn)
ATL08_filt_tindex_master =   pd.read_csv(ATL08_filt_tindex_master_fn)
ATL08_filt_sample_tindex_master = pd.read_csv(ATL08_filt_sample_tindex_master_fn)

In [3]:
UPDATE_TINDEX = False

if not os.path.isfile(tindex_master_fn):
     UPDATE_TINDEX = True
else:
    print('Using existing tindex')
    print(tindex_master_fn)

    
if UPDATE_TINDEX:
    print(f"Building master tile index for: {DPS_DATA_TYPE}")
    dps_month = 'run_LC_height_thresholds'
    d_min = 1
    #os.system(f"python /projects/icesat2_boreal/lib/build_tindex_master.py --type {DPS_DATA_TYPE} -m {dps_month} -d_min {d_min}")
    !python /projects/icesat2_boreal/lib/build_tindex_master_v2.py --type $DPS_DATA_TYPE -m $dps_month -d_min $d_min


Building master tile index for: ATL08_filt
python: can't open file '/projects/icesat2_boreal/lib/build_tindex_master_v2.py': [Errno 2] No such file or directory


### Get the tiles needed for the data of interest

In [4]:
# Build up a dataframe from the list of dps output files
tindex_master = pd.read_csv(tindex_master_fn)

if DPS_DATA_TYPE == 'Landsat' or DPS_DATA_TYPE == 'HLS' or DPS_DATA_TYPE == 'Topo':
    DPS_DATA_USER = 'nathanmthomas'
    
tindex_master['s3'] = [local_to_s3(local_path, user=DPS_DATA_USER, type = 'private') for local_path in tindex_master['local_path']]
print(f"# of expected {DPS_DATA_TYPE} tiles:\t\t\t\t\t\t{len(tindex_master)}")

# Wherever there is a 90k Topo tile, there should be data...
# Get all covar tiles that should account for the set of output we want
tiles_covars = pd.read_csv(Topo_tindex_master_fn).tile_num
print(f"# of expected covar (from Topo) tiles:\t\t\t\t\t{len(tiles_covars)}")

# Get all boreal tiles
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg' 
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
#boreal_tile_index.astype({'layer':'int'})
#boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.
# For some reason, doing this causes 'MosaicJSON.from_features()' to fail...(below)
if True:
    # Remove bad tiles
    boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
    
select_needs = [3360,2994,3190,2840,3012,3014,3017,2932,1261,1263,1264,988,978,794, 380,378,411,821,861,
                812,765,764,1308,1302,1469,1406,2495,2883,2965,3321,3509,3510,3327,3335,2976,2906,2907,2894,2814,4253,4293,4403,4440,4408,4372,4477,3986]
tile_matches_select_needs = boreal_tile_index.merge(ATL08_filt_tindex_master[ATL08_filt_tindex_master['tile_num'].isin(select_needs)], how='right', on='tile_num')
print(f"# of {DPS_DATA_TYPE} tiles needed (compared with {os.path.basename(ATL08_filt_tindex_master_fn)}):\t{len(tile_matches_select_needs)}")

print(f"# of total tiles supposedly needed (from {os.path.basename(boreal_tile_index_path)}):\t{len(boreal_tile_index)}")

if len(tile_matches_select_needs)<100:
    print([t for t in tile_matches_select_needs.tile_num])


OSError: [Errno 107] Transport endpoint is not connected: '/projects/my-private-bucket/dps_output/run_tile_atl08_ubuntu/master/2022/run_LC_height_thresholds_v2/ATL08_filt_tindex_master.csv'

### Identify duplicate tiles

In [ ]:
duplicate_tiles = [item for item, count in collections.Counter(tindex_master["tile_num"]).items() if count > 1]
print(duplicate_tiles)

### Identify completed, missing, failed, & duplicate tiles

In [ ]:
print(f"Tile status report for {DPS_DATA_TYPE} from {tindex_master_fn}:")
# For the tindex_master, convert it into vector tiles that show the tiles we have
# Select the rows we have results for
tile_index_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(bad_tiles)][['tile_num','s3_path','local_path']], how='left', on='tile_num')
#tile_matches_atl08_filt_samples = boreal_tile_index.merge(ATL08_filt_sample_tindex_master[~ATL08_filt_sample_tindex_master['tile_num'].isin(bad_tiles)], how='right', on='tile_num')
print(f'\t# tiles matched:\t\t{len(tile_index_matches)}')

# Use the Topo tiles (COVAR set) to get the diff of what tiles we have and what we want
# MISSING TILES = DIFF(tiles_we_want, tiles_we_have)
#tile_nums_missing = np.setdiff1d(tiles_covars, tile_index_matches.tile_num)
tile_nums_missing_list = list(set(tiles_covars) - set(tile_index_matches.tile_num))
tile_nums_missing_list = list(set(tile_index_matches.tile_num) - set(tiles_covars))
# The ATL08 tiles we have may not exactly match the set of COVAR tiles we have
# Use the set of ATL08 tiles we have, crossed with the tiles we're missing (based on COVAR set), to get tiles we're missing that we should definitely proces (b/c we have both COVAR and ATL08 tiles for them)
#tile_index_missing = boreal_tile_index.merge(boreal_tile_index[boreal_tile_index['tile_num'].isin(tile_nums_missing_list)], how='right', on='tile_num')
#tile_index_missing = boreal_tile_index.merge(tindex_master[tindex_master['tile_num'].isin(tile_nums_missing_list)], how='inner', on='tile_num')
tile_index_missing = boreal_tile_index[boreal_tile_index['tile_num'].isin(tile_nums_missing_list)]

print(f'\t# tiles missing:\t\t{len(tile_index_missing)}')
tile_index_missing.to_file(f'/projects/my-public-bucket/DPS_tile_lists/Need_{DPS_DATA_TYPE}_tindex_master.gpkg', driver='GPKG')
#print(tile_index_missing.head())

# Meh, this doesnt give us fails
#tile_matches_failed = boreal_tile_index.merge(Topo_tindex_master[Topo_tindex_master['tile_num'].isin(tile_nums_missing)], how='right', on='tile_num')
#print(f'Missing b/c failed: \t{len(tile_matches_failed)}')

# Duplicates are also removed in build_tindex_master
tile_matches_duplicates = boreal_tile_index.merge(tiles_covars[tiles_covars.isin(duplicate_tiles)], how='right', on='tile_num')
print(f'\t# tiles duplicated: \t\t{len(tile_matches_duplicates)}')

# Drop duplicates
tile_index_matches = tile_index_matches.drop_duplicates(subset=['tile_num'], keep='last')
print(f"Missing tiles:\t{tile_nums_missing_list}")
print(f"Bad tiles:\t{bad_tiles}")

ax = tile_index_matches.plot(color='black')
tile_index_missing.plot(column='tile_group', ax=ax)


In [ ]:
if 's3_path' in tile_index_matches.columns:
    tile_index_matches['s3'] = tile_index_matches['s3_path']
    
tile_index_matches = tile_index_matches[tile_index_matches['s3'].notna()]
tile_index_matches.head()

## Build a MosaicJSON

##### Build tile geojsons needed for mosaic jsons

In [ ]:
tile_matches_geojson_string = tile_index_matches.to_crs("EPSG:4326").to_json()
tile_matches_geojson = json.loads(tile_matches_geojson_string)
#Write copy to disk for debug
tile_matches_geojson_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master.json' 
tile_index_matches.to_file(tile_matches_geojson_fn, driver='GeoJSON')

if len(tile_index_missing) > 0:
    tile_index_missing_geojson_string = tile_index_missing.to_crs("EPSG:4326").to_json()
    tile_index_missing_geojson = json.loads(tile_index_missing_geojson_string)


In [ ]:
!pip install pystac

In [ ]:
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

out_mosaic_json_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master_mosaic.json' 
#out_mosaic_json_fn = f's3://maap-ops-workspace/shared/alexdevseed/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master_mosaic.json' 

print(f"Building {out_mosaic_json_fn}")
mosaicdata = MosaicJSON.from_features(tile_matches_geojson.get('features'), minzoom=6, maxzoom=18, accessor=get_accessor)

with MosaicBackend(out_mosaic_json_fn, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

## View the Results with Folium

In [ ]:
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker, features, Figure, CircleMarker
from folium import plugins

# Setup the mosaic tiling
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/" #titiler.maap-project.org
tiler_mosaic =  "".join([tiler_base, "mosaicjson/tiles/{z}/{x}/{y}"])

import branca.colormap as cm
import matplotlib.cm
pal_height_cmap = cm.LinearColormap(colors = ['black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'], vmin=0, vmax=10)
pal_height_cmap.caption = 'Vegetation height from  ATL08 @ 30 m (h_can; rh98)'
#pal_height_cmap

dem_tiles_index_path = '/projects/shared-buckets/nathanmthomas/dem30m_tiles.geojson'
dem_tiles_index = geopandas.read_file(dem_tiles_index_path)
dem_tiles_index['tile_num'] = dem_tiles_index.index
dem_tiles_index.to_file('/projects/my-public-bucket/dem30m_tiles_v2.geojson')

In [ ]:
maplib_folium.MAP_DPS_RESULTS(tiler_mosaic, DPS_DATA_TYPE, boreal_tile_index, tile_index_matches, tile_index_missing, 
                              mosaic_json_dict = {
                                        'agb_mosaic_json_s3_fn':    's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/AGB_tindex_master_mosaic.json',
                                        'topo_mosaic_json_s3_fn':   's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master_mosaic.json',
                                        'mscomp_mosaic_json_s3_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS_tindex_master_mosaic.json'
                                    },
                             MS_BANDMAX=1,
                             MS_BANDNUM=7,
                             MS_BANDCOLORBAR='plasma'
                             )

In [ ]:
maplib_folium.MAP_DPS_RESULTS(tiler_mosaic, DPS_DATA_TYPE, dem_tiles_index, tile_index_matches, tile_index_missing)

In [ ]:
maplib_folium.map_tile_n_obs(tindex_master_fn='s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv', 
                   map_name = '# of filtered ATL08 obs. from night & day', max_n_obs=15000)

In [ ]:
maplib_folium.map_tile_n_obs(tindex_master_fn='s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_sample_tindex_master.csv', 
                   map_name = '# of samples of filtered ATL08 obs. from night', max_n_obs=250)

In [ ]:
maplib_folium.map_tile_atl08(2804, DO_NIGHT=True)